In [ ]:
import json
import pandas as pd
import openai
from typing import Dict, List, Tuple
import os
import time
from datetime import datetime
import logging

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [8]:

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class QualitativeAnalyzer:
    def __init__(self, api_key: str):
        """
        정성적 분석을 위한 클래스 초기화

        Args:
            api_key: OpenAI API 키
        """
        self.client = openai.OpenAI(api_key=api_key)
        self.batch_requests = []

    def load_search_results(self, folder_path: str) -> Dict:
        """
        search_results_*.json 파일들을 모두 로드

        Args:
            folder_path: 검색 결과 JSON 파일들이 있는 폴더 경로

        Returns:
            모든 검색 결과를 통합한 딕셔너리
        """
        all_results = {}

        # search_results_*.json 파일들 찾기
        for i in range(1, 10):  # 1~9까지 확인
            file_path = os.path.join(folder_path, f"search_results_{i}.json")
            if os.path.exists(file_path):
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        all_results.update(data)
                        logger.info(f"Loaded {len(data)} records from {file_path}")

                        # 첫 번째 항목의 구조 확인 (디버깅용)
                        if i == 1 and data:
                            first_key = list(data.keys())[0]
                            first_item = data[first_key]
                            logger.info(f"Sample data structure for {first_key}:")
                            logger.info(f"Keys: {list(first_item.keys())}")
                            if 'results' in first_item:
                                logger.info(f"Results type: {type(first_item['results'])}")
                                if isinstance(first_item['results'], dict):
                                    logger.info(f"Results dict keys: {list(first_item['results'].keys())}")

                except Exception as e:
                    logger.error(f"Error loading {file_path}: {e}")

        logger.info(f"Total loaded records: {len(all_results)}")
        return all_results

    def create_analysis_prompt(self, company_info: Dict) -> str:
        """
        정성적 분석을 위한 프롬프트 생성

        Args:
            company_info: 기업 정보 딕셔너리

        Returns:
            분석 프롬프트 문자열
        """
        corp_code = company_info.get('corp_code', 'Unknown')
        corp_name = "Unknown"

        # 쿼리에서 기업명 추출 시도
        query = company_info.get('query', '')
        if query:
            corp_name = query.split()[0] if query.split() else "Unknown"

        # JSON 구조에 맞게 검색 결과 접근: company_info['results']['results']
        results_data = company_info.get('results', {})
        if isinstance(results_data, dict):
            search_results = results_data.get('results', [])
        else:
            search_results = []
            logger.warning(f"Unexpected results structure for corp_code {corp_code}: {type(results_data)}")

        # 검색 결과에서 내용 추출
        content_summary = ""
        if isinstance(search_results, list) and search_results:
            for i, result in enumerate(search_results[:5]):  # 상위 5개 결과만 사용
                if isinstance(result, dict):
                    title = result.get('title', '')
                    content = result.get('content', '')
                    url = result.get('url', '')

                    if title and content and content != "Missing":  # "Missing" 내용은 제외
                        content_summary += f"{i+1}. {title}\n   내용: {content}\n   출처: {url}\n\n"

        # 검색 결과가 없거나 유효하지 않은 경우 처리
        if not content_summary.strip():
            content_summary = f"기업코드 {corp_code}({corp_name})에 대한 유효한 검색 결과가 부족합니다."

        prompt = f"""
다음은 기업 코드 {corp_code}({corp_name})에 대한 검색 결과입니다. 이 정보를 바탕으로 신용등급 평가를 위한 정성적 요인들을 분석해주세요.

검색 결과 내용:
{content_summary}

위 정보를 분석하여 다음 형태의 JSON으로 응답해주세요:

{{
    "positive_factors": [
        "구체적인 긍정적 요인 1",
        "구체적인 긍정적 요인 2"
    ],
    "negative_factors": [
        "구체적인 부정적 요인 1",
        "구체적인 부정적 요인 2"
    ]
}}

분석 기준:
1. 긍정적 요인:
   - 매출 성장, 영업이익 개선, 시장 점유율 확대
   - 신규 사업 진출, 기술력 강화, 파트너십 확대
   - 재무구조 개선, 부채 감소, 유동성 증가
   - 시장 지위 강화, 브랜드 가치 향상

2. 부정적 요인:
   - 매출 감소, 영업손실, 시장 점유율 하락
   - 경쟁 심화, 규제 리스크, 사업 환경 악화
   - 부채 증가, 유동성 부족, 재무 악화
   - 경영진 교체, 법적 리스크, 평판 손상

3. 주의사항:
   - 각 요인은 검색 결과에 기반하여 구체적으로 작성
   - 추측이나 일반론은 피하고 사실에 기반한 내용만 포함
   - 정보가 부족하면 해당 배열을 비워두기
   - 각 배열에 최대 3-5개 요인만 포함

반드시 유효한 JSON 형태로만 응답하고 다른 설명은 포함하지 마세요.
"""
        return prompt

    def prepare_batch_requests(self, search_results: Dict, model: str = "gpt-4o-mini") -> List[Dict]:
        """
        Batch API 요청을 위한 요청 리스트 준비

        Args:
            search_results: 검색 결과 딕셔너리
            model: 사용할 OpenAI 모델

        Returns:
            Batch API 요청 리스트
        """
        batch_requests = []

        for corp_code, company_info in search_results.items():
            prompt = self.create_analysis_prompt(company_info)

            request = {
                "custom_id": f"analysis_{corp_code}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": model,
                    "messages": [
                        {
                            "role": "system",
                            "content": "당신은 기업 신용분석 전문가입니다. 주어진 정보를 바탕으로 정확하고 객관적인 정성적 요인 분석을 수행합니다."
                        },
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                    "max_tokens": 1000,
                    "temperature": 0.3
                }
            }
            batch_requests.append(request)

        return batch_requests

    def create_batch_file(self, batch_requests: List[Dict], filename: str = "batch_requests.jsonl") -> str:
        """
        Batch API용 JSONL 파일 생성

        Args:
            batch_requests: 배치 요청 리스트
            filename: 저장할 파일명

        Returns:
            생성된 파일의 경로
        """
        with open(filename, 'w', encoding='utf-8') as f:
            for request in batch_requests:
                f.write(json.dumps(request, ensure_ascii=False) + '\n')

        logger.info(f"Created batch file: {filename} with {len(batch_requests)} requests")
        return filename

    def submit_batch(self, batch_file_path: str) -> str:
        """
        OpenAI Batch API에 작업 제출

        Args:
            batch_file_path: 배치 파일 경로

        Returns:
            배치 작업 ID
        """
        # 파일 업로드
        with open(batch_file_path, 'rb') as f:
            batch_input_file = self.client.files.create(
                file=f,
                purpose="batch"
            )

        # 배치 작업 생성
        batch = self.client.batches.create(
            input_file_id=batch_input_file.id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
                "description": "Qualitative analysis for credit rating"
            }
        )

        logger.info(f"Batch submitted with ID: {batch.id}")
        return batch.id

    def check_batch_status(self, batch_id: str) -> Dict:
        """
        배치 작업 상태 확인

        Args:
            batch_id: 배치 작업 ID

        Returns:
            배치 상태 정보
        """
        batch = self.client.batches.retrieve(batch_id)
        return {
            "id": batch.id,
            "status": batch.status,
            "completed_at": batch.completed_at,
            "failed_at": batch.failed_at,
            "request_counts": batch.request_counts.__dict__ if batch.request_counts else None
        }

    def download_batch_results(self, batch_id: str, output_filename: str = "batch_results.jsonl") -> str:
        """
        배치 작업 결과 다운로드

        Args:
            batch_id: 배치 작업 ID
            output_filename: 결과 저장 파일명

        Returns:
            다운로드된 파일 경로
        """
        batch = self.client.batches.retrieve(batch_id)

        if batch.status != "completed":
            raise Exception(f"Batch not completed. Status: {batch.status}")

        # 결과 파일 다운로드
        result_file_id = batch.output_file_id
        result = self.client.files.content(result_file_id)

        with open(output_filename, 'wb') as f:
            f.write(result.content)

        logger.info(f"Downloaded batch results to: {output_filename}")
        return output_filename

    def parse_batch_results(self, results_file_path: str) -> Dict[str, Dict]:
        """
        배치 결과 파일 파싱

        Args:
            results_file_path: 결과 파일 경로

        Returns:
            파싱된 결과 딕셔너리
        """
        parsed_results = {}

        with open(results_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                result = json.loads(line.strip())
                custom_id = result['custom_id']
                corp_code = custom_id.replace('analysis_', '')

                try:
                    # GPT 응답에서 JSON 추출
                    response_content = result['response']['body']['choices'][0]['message']['content']
                    analysis_data = json.loads(response_content)

                    parsed_results[corp_code] = {
                        'positive_factors': analysis_data.get('positive_factors', []),
                        'negative_factors': analysis_data.get('negative_factors', [])
                    }

                except Exception as e:
                    logger.error(f"Error parsing result for {corp_code}: {e}")
                    parsed_results[corp_code] = {
                        'positive_factors': [],
                        'negative_factors': []
                    }

        return parsed_results

    def integrate_with_financial_data(self, csv_path: str, analysis_results: Dict[str, Dict],
                                    output_path: str = "integrated_financial_data.csv") -> pd.DataFrame:
        """
        재무 데이터와 정성적 분석 결과 통합

        Args:
            csv_path: 원본 재무 데이터 CSV 경로
            analysis_results: 정성적 분석 결과
            output_path: 통합 결과 저장 경로

        Returns:
            통합된 DataFrame
        """
        # 원본 재무 데이터 로드
        df = pd.read_csv(csv_path)
        logger.info(f"Loaded financial data: {len(df)} records")

        # 정성적 요인 컬럼 추가
        df['positive_factors'] = ''
        df['negative_factors'] = ''

        # 분석 결과와 매칭하여 데이터 추가
        matched_count = 0
        for idx, row in df.iterrows():
            corp_code = str(row['corp_code'])

            if corp_code in analysis_results:
                positive_factors = analysis_results[corp_code]['positive_factors']
                negative_factors = analysis_results[corp_code]['negative_factors']

                # 리스트를 문자열로 변환 (JSON 배열 형태로)
                df.at[idx, 'positive_factors'] = json.dumps(positive_factors, ensure_ascii=False)
                df.at[idx, 'negative_factors'] = json.dumps(negative_factors, ensure_ascii=False)

                matched_count += 1

        logger.info(f"Matched {matched_count}/{len(df)} records with analysis results")

        # 결과 저장
        df.to_csv(output_path, index=False, encoding='utf-8')
        logger.info(f"Integrated data saved to: {output_path}")

        return df


def main():
    """
    메인 실행 함수
    """
    # OpenAI API 키 설정 (환경변수에서 가져오거나 직접 입력)
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        api_key = input("OpenAI API 키를 입력하세요: ")

    # 경로 설정
    folder_path = "D:/JetBrains/ai/add-data-real-company"
    csv_path = os.path.join(folder_path, "dart_general_company_financial_fixed.csv")

    # 분석기 초기화
    analyzer = QualitativeAnalyzer(api_key)

    try:
        # 1. 검색 결과 로드
        logger.info("Loading search results...")
        search_results = analyzer.load_search_results(folder_path)

        if not search_results:
            logger.error("No search results found!")
            return

        # 2. 배치 요청 준비
        logger.info("Preparing batch requests...")
        batch_requests = analyzer.prepare_batch_requests(search_results, model="gpt-4o-mini")

        # 3. 배치 파일 생성
        batch_file = analyzer.create_batch_file(batch_requests, "qualitative_analysis_batch.jsonl")

        # 4. 배치 작업 제출
        logger.info("Submitting batch to OpenAI...")
        batch_id = analyzer.submit_batch(batch_file)
        print(f"Batch ID: {batch_id}")
        print("배치 작업이 제출되었습니다. 완료까지 시간이 걸릴 수 있습니다.")

        # 5. 상태 확인 (완료될 때까지 대기)
        logger.info("Waiting for batch completion...")
        while True:
            status = analyzer.check_batch_status(batch_id)
            print(f"Status: {status['status']}")

            if status['status'] == 'completed':
                logger.info("Batch completed successfully!")
                break
            elif status['status'] in ['failed', 'cancelled']:
                logger.error(f"Batch failed with status: {status['status']}")
                return

            time.sleep(60)  # 1분마다 상태 확인

        # 6. 결과 다운로드
        logger.info("Downloading batch results...")
        results_file = analyzer.download_batch_results(batch_id, "qualitative_results.jsonl")

        # 7. 결과 파싱
        logger.info("Parsing batch results...")
        analysis_results = analyzer.parse_batch_results(results_file)

        # 8. 재무 데이터와 통합
        logger.info("Integrating with financial data...")
        integrated_df = analyzer.integrate_with_financial_data(
            csv_path,
            analysis_results,
            os.path.join(folder_path, "integrated_financial_qualitative_data.csv")
        )

        print(f"완료! 통합된 데이터: {len(integrated_df)} 행")
        print("새로운 컬럼:")
        print("- positive_factors: 긍정적 요인들")
        print("- negative_factors: 부정적 요인들")

    except Exception as e:
        logger.error(f"Error in main process: {e}")
        raise


# 배치가 이미 실행 중인 경우 상태 확인 및 결과 처리 함수
def process_existing_batch(batch_id: str, api_key: str, folder_path: str):
    """
    기존 배치 작업의 결과를 처리하는 함수

    Args:
        batch_id: 기존 배치 작업 ID
        api_key: OpenAI API 키
        folder_path: 작업 폴더 경로
    """
    analyzer = QualitativeAnalyzer(api_key)
    csv_path = os.path.join(folder_path, "dart_general_company_financial_fixed.csv")

    try:
        # 상태 확인
        status = analyzer.check_batch_status(batch_id)
        print(f"Batch Status: {status}")

        if status['status'] == 'completed':
            # 결과 다운로드 및 처리
            results_file = analyzer.download_batch_results(batch_id, "qualitative_results.jsonl")
            analysis_results = analyzer.parse_batch_results(results_file)

            # 재무 데이터와 통합
            integrated_df = analyzer.integrate_with_financial_data(
                csv_path,
                analysis_results,
                os.path.join(folder_path, "integrated_financial_qualitative_data.csv")
            )

            print(f"완료! 통합된 데이터: {len(integrated_df)} 행")

    except Exception as e:
        logger.error(f"Error processing existing batch: {e}")


if __name__ == "__main__":
    main()

2025-06-20 00:29:01,003 - INFO - Loading search results...
2025-06-20 00:29:01,060 - INFO - Loaded 500 records from D:/JetBrains/ai/add-data-real-company\search_results_1.json
2025-06-20 00:29:01,062 - INFO - Sample data structure for 33790.0:
2025-06-20 00:29:01,063 - INFO - Keys: ['query', 'corp_code', 'timestamp', 'results']
2025-06-20 00:29:01,064 - INFO - Results type: <class 'dict'>
2025-06-20 00:29:01,064 - INFO - Results dict keys: ['query', 'follow_up_questions', 'answer', 'images', 'results', 'response_time']
2025-06-20 00:29:01,103 - INFO - Loaded 500 records from D:/JetBrains/ai/add-data-real-company\search_results_2.json
2025-06-20 00:29:01,145 - INFO - Loaded 500 records from D:/JetBrains/ai/add-data-real-company\search_results_3.json
2025-06-20 00:29:01,182 - INFO - Loaded 500 records from D:/JetBrains/ai/add-data-real-company\search_results_4.json
2025-06-20 00:29:01,188 - INFO - Loaded 59 records from D:/JetBrains/ai/add-data-real-company\search_results_5.json
2025-06-

Batch ID: batch_68542cbf980c819080143b8a1a906b4a
배치 작업이 제출되었습니다. 완료까지 시간이 걸릴 수 있습니다.


2025-06-20 00:29:04,251 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_68542cbf980c819080143b8a1a906b4a "HTTP/1.1 200 OK"


Status: validating


2025-06-20 00:30:04,543 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_68542cbf980c819080143b8a1a906b4a "HTTP/1.1 200 OK"


...

Status: finalizing


2025-06-20 02:41:45,984 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_68542cbf980c819080143b8a1a906b4a "HTTP/1.1 200 OK"
2025-06-20 02:41:45,987 - INFO - Batch completed successfully!
2025-06-20 02:41:45,989 - INFO - Downloading batch results...


Status: completed


2025-06-20 02:41:46,210 - INFO - HTTP Request: GET https://api.openai.com/v1/batches/batch_68542cbf980c819080143b8a1a906b4a "HTTP/1.1 200 OK"
2025-06-20 02:41:46,762 - INFO - HTTP Request: GET https://api.openai.com/v1/files/file-ALMuidjFsEYjM7KswSyjGa/content "HTTP/1.1 200 OK"
2025-06-20 02:41:47,512 - INFO - Downloaded batch results to: qualitative_results.jsonl
2025-06-20 02:41:47,513 - INFO - Parsing batch results...
2025-06-20 02:41:47,573 - INFO - Integrating with financial data...
2025-06-20 02:41:47,619 - INFO - Loaded financial data: 2059 records
2025-06-20 02:41:47,862 - INFO - Matched 2058/2059 records with analysis results
2025-06-20 02:41:47,966 - INFO - Integrated data saved to: D:/JetBrains/ai/add-data-real-company\integrated_financial_qualitative_data.csv


완료! 통합된 데이터: 2059 행
새로운 컬럼:
- positive_factors: 긍정적 요인들
- negative_factors: 부정적 요인들
